Nomes: Bel Cogo e Bruno da Siqueira Hoffmann.

Professor: Felipe de Morais.

Disciplina: Ciência de Dados e Big Data.

# Tarefa 5

O banco de dados utilizado para essa tarefa contém dados referentes à dados de transações de um marketplace. O banco de dados consiste em 5 tabelas, sendo elas:

- sellers (vendedores): Contém informações sobre os vendedores, incluindo o estado em que estão localizados.
- products (produtos): Detalha os produtos disponíveis no marketplace, incluindo categorias e pesos.
- orders (ordens): Fornece informações básicas sobre cada pedido, como registros de data/hora e contato do cliente.
- order_items (items ordem): Agrupa os itens associados a cada pedido, incluindo detalhes de preço.
- order_reviews (avaliação ordens): Armazena as avaliações relacionadas a cada pedido, incluindo classificações numéricas e comentários em texto.

Essa base de dados foi encontrada através da plataforma Kaggle, através do link: https://www.kaggle.com/datasets/petewojtczak/raw-transactional-data.

É possível verificar o ER do banco de dados abaixo:

![Diagrama ER.jpg](./Diagrama%20ER.jpg)

Para importar o banco de dados foi executado o comando abaixo:

In [7]:
import pandas as pd
import sqlite3

connection = sqlite3.connect('transactional_data.sqlite')

## Pergunta 1

Pergunta: Quais são os sellers que possuem um faturamento inferior a R$ 100? E qual é o seu faturamento? Listando os 10 com menor faturamento e o seu estado.

Resposta:

In [8]:
query = """
  SELECT
    s.seller_id as identificador_vendedor,
    SUM(oi.price) as faturamento,
    s.seller_state as estado_do_vendedor
  FROM sellers s
    INNER JOIN order_items oi
      ON s.seller_id = oi.seller_id
  GROUP BY s.seller_id
  HAVING faturamento < 100
  ORDER BY faturamento ASC
  LIMIT 10;
"""

df = pd.read_sql(query, connection)
df

,identificador_vendedor,faturamento,estado_do_vendedor
0,77128dec4bec4878c37ab7d6169d6f26,6.50,SP
1,1fa2d3def6adfa70e58c276bb64fe5bb,6.90,SP
2,702835e4b785b67a084280efca355756,7.60,MG
3,ad14615bdd492b01b0d97922e87cb87f,8.25,SC
4,4965a7002cca77301c82d3f91b82e1a9,8.49,SP
5,0f94588695d71662beec8d883ffacf09,9.00,SC
6,c18309219e789960add0b2255ca4b091,9.90,RJ
7,95cca791657aabeff15a07eb152d7841,9.99,PR
8,344223b2a90784f64136a8a5da012e7f,10.90,SC
9,f9f68daa2af419d38f745478bcb3da1a,11.20,SP


## Questão 2

Pergunta: Quais os 5 produtos (sua categoria e identificador) mais vendidos com valor inferior a 50.0 e superior a 40.0? Qual é a quantidade de vendas deles? E qual é o valor deles?

Resposta:

In [9]:
query = """
  SELECT 
    COUNT(1) as quantidade_de_vendas,
    p.product_id as identificador_produto,
    oi.price as valor,
    p.product_category_name as categoria
  FROM products p
  INNER JOIN order_items oi
  ON
    oi.product_id = p.product_id
  WHERE
    oi.price <= 50.0
    AND oi.price >= 40.0
  GROUP BY p.product_id
  ORDER BY quantidade_de_vendas DESC
  LIMIT 5;
"""

df = pd.read_sql(query, connection)
df

,quantidade_de_vendas,identificador_produto,valor,categoria
0,166,368c6c730842d78016ad823897a372db,49.9,ferramentas_jardim
1,154,422879e10f46682990de24d770e7f83d,49.0,ferramentas_jardim
2,148,53759a2ecddad2bb87a079a1f1519f73,49.9,ferramentas_jardim
3,146,389d119b48cf3043d311335e499d9c6b,49.9,ferramentas_jardim
4,139,d017a2151d543a9885604dc62a3d9dcc,49.0,fashion_bolsas_e_acessorios


## Questão 3

Pergunta: Quantas vendas o estado do Rio de Janeiro e o estado de São Paulo possuem? E qual é o valor total faturado por cada estado?

Resposta:

In [10]:
query = """
  SELECT 
    COUNT(1) as quantidade_venda,
    s.seller_state as UF,
    SUM(oi.price) as faturamento
  FROM order_items oi
    INNER JOIN sellers s
      ON oi.seller_id = s.seller_id
  WHERE
    s.seller_state = 'RJ'
    OR s.seller_state = 'SP'
  GROUP BY s.seller_state
  ORDER BY quantidade_venda DESC;
"""

df = pd.read_sql(query, connection)
df

,quantidade_venda,UF,faturamento
0,78604,SP,8.509511e+06
1,4685,RJ,8.206116e+05


## Questão 4

Pergunta: Quais os 5 sellers com mais pedidos com review ruim? E qual é a média de nota deles?

Resposta:

In [11]:
query = """
  SELECT 
    sl.seller_id as id_vendedor,
    AVG(orw.review_score) as media
  FROM sellers sl
  INNER JOIN order_items oi
    ON oi.seller_id = sl.seller_id
  INNER JOIN order_reviews orw
    ON orw.order_id = oi.order_id
  GROUP BY id_vendedor
  ORDER BY media ASC
  LIMIT 5;
"""

df = pd.read_sql(query, connection)
df

,id_vendedor,media
0,0015a82c2db000af6aaaf3ae2ecb0532,1.0
1,010da0602d7774602cd1b3f5fb7b709e,1.0
2,0791d9fc1e30678ecf03d3e55fa108d3,1.0
3,0aa2205ca24f113f4658a5c536667426,1.0
4,0dfbed20065e425d2eaefb101f9816c0,1.0


## Questão 5

Pergunta: Quais são os 10 produtos (sua categoria e seu identificador) com mais ordens? Qual foi a quantidade de venda deles? E quanto eles faturam?

Resposta:

In [12]:
query = """
  SELECT 
    COUNT(1) as quantidade,
    p.product_id as identificador_produto, 
    p.product_category_name as nome_categoria,
    SUM(oi.price) as preco_total
  FROM products p 
    INNER JOIN order_items oi 
      on p.product_id = oi.product_id 
  GROUP BY oi.product_id 
  ORDER BY quantidade DESC
  LIMIT 10;
"""

df = pd.read_sql(query, connection)
df

,quantidade,identificador_produto,nome_categoria,preco_total
0,520,aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,37104.30
1,484,422879e10f46682990de24d770e7f83d,ferramentas_jardim,26577.22
2,477,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,42049.66
3,390,389d119b48cf3043d311335e499d9c6b,ferramentas_jardim,21336.79
4,388,368c6c730842d78016ad823897a372db,ferramentas_jardim,21056.80
5,373,53759a2ecddad2bb87a079a1f1519f73,ferramentas_jardim,20387.20
6,332,d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,45620.56
7,321,53b36df67ebb7c41585e8d54d6772e08,relogios_presentes,37454.63
8,274,154e7e31ebfa092203795c972e5804a6,beleza_saude,6173.26
9,272,3dd2a17168ec895c781a9191c1e95ad7,informatica_acessorios,40782.80
